# Librerías

In [ ]:
#!pip install pysqlite3
#!pip install scikit-lear
!pip install numpy==1.23.5
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505213 sha256=35ec2e5e83730071abe13f573d229fbc125ddfa0485b5f07541bee2d5476825f
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
from collections import Counter
from google.colab import drive
from google.colab import files
import os
import sys
from ipywidgets import interact
from sklearn import neighbors
import joblib
#from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

#Librerias estadísticas y de pruebas
import scipy.stats as stats
from scipy.stats import gaussian_kde
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler

import datetime

In [ ]:
from surprise import Reader, Dataset

# Conectar con google drive


In [ ]:
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
#drive.flush_and_unmount()  #Linea en caso de tener que desconectar el drive por algún tipo de falla
drive.mount('/content/drive') #Linea para conectar al drive

Mounted at /content/drive


In [ ]:
path="/content/drive/MyDrive/analitica 3/sistemas_recomendacion" ### ruta del repositorio en drive
os.chdir(path) ### volver la carpeta del repositorio directorio de trabajo
sys.path.append(path) ### agregarla al path, poder leer archivos de funciones propios como paquetes

In [ ]:
import a_funciones as fn


# Base de datos

In [ ]:
conn = sql.connect('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/data/db_movies3')
cur = conn.cursor()

In [ ]:
# Creación de cursor para  ejecutar consultas en la base de datos
# Visualizar las tablas contenidas en la base de datos
cur.execute("SELECT name FROM sqlite_master where type='table'")
cur.fetchall()

[('movies_final',), ('ratings_final',), ('df_final',), ('df_terminado',)]

#Modelos

##Sistema de recomendación basado en contenido KNN (Basado en el usuario)

In [ ]:
df_modelos=joblib.load('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_modelos.joblib')
df_final=joblib.load('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_final.joblib')

In [ ]:
#### seleccionar usuario para recomendaciones ####
usuarios=pd.read_sql('select distinct (user_id) as user_id from ratings_final',conn)

In [ ]:
user_id=500 ### para ejemplo manual

In [ ]:
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select *from ratings_final where user_id=:user',conn, params={'user':user_id})

    ###convertir ratings del usuario a array
    l_books_r=ratings['movie_id'].to_numpy()

    ###agregar la columna de isbn y titulo del libro a dummie para filtrar y mostrar nombre
    df_modelos[['movie_id','title']]=df_final[['movie_id','title']]

    ### filtrar libros calificados por el usuario
    books_r=df_modelos[df_modelos['movie_id'].isin(l_books_r)]

    ## eliminar columna nombre e isbn
    books_r=books_r.drop(columns=['movie_id','title'])
    books_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
    centroide=books_r.groupby("indice").mean()


    ### filtrar libros no leídos
    books_nr=df_modelos[~df_modelos['movie_id'].isin(l_books_r)]
    ## eliminbar nombre e isbn
    books_nr=books_nr.drop(columns=['movie_id','title'])

    ### entrenar modelo
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(books_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=df_final.loc[ids][['title','movie_id']]
    leidos=df_final[df_final['movie_id'].isin(l_books_r)][['title','movie_id']]

    return recomend_b


recomendar(500)

,title,movie_id
7561,Avengers: Age of Ultron (2015),122892
8760,"Firm, The (1993)",454
45907,Glory (1989),1242
8634,Collateral (2004),8798
45315,Anaconda (1997),1499
19601,Cool Hand Luke (1967),1276
31340,GoldenEye (1995),10
52044,Black Swan (2010),81591
31293,Reservoir Dogs (1992),1089
22089,Strange Brew (1983),5650


In [ ]:
print(interact(recomendar))

##Sistema de recomendación filtro colaborativo

In [ ]:
ratings=pd.read_sql('select * from ratings_final', conn)
ratings.head()

,user_id,movie_id,rating,year_ratings,month,day
0,1,1,4.0,2000,7,30
1,1,3,4.0,2000,7,30
2,1,6,4.0,2000,7,30
3,1,47,5.0,2000,7,30
4,1,50,5.0,2000,7,30


In [ ]:
reader = Reader(rating_scale=(1, 10))
data   = Dataset.load_from_df(ratings[['user_id','movie_id','rating']], reader)

In [ ]:
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}

In [ ]:
model=models[1]
for model in models:

    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)

    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.642606,0.840621,0.354452,2.880362
knns.KNNWithZScore,0.647561,0.850633,0.302817,2.411425
knns.KNNWithMeans,0.652762,0.852678,0.197440,2.482794
knns.KNNBasic,0.693324,0.902772,0.218472,1.951754


In [ ]:
param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5,2], \
                                'user_based': [False, True]}
             }

In [ ]:
gridsearchKNNBaseline = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=-1)

gridsearchKNNBaseline.fit(data)

In [ ]:
gridsearchKNNBaseline.best_params["rmse"]
gridsearchKNNBaseline.best_score["rmse"]
gs_model=gridsearchKNNBaseline.best_estimator['rmse']

In [ ]:
trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrnamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
#joblib.dump(model,'/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/recommodel.joblib')

In [ ]:
model.predict(uid=5, iid='5',r_ui='')

Prediction(uid=5, iid='5', r_ui='', est=3.5093755789717713, details={'was_impossible': False})

In [ ]:
predset = trainset.build_anti_testset()
len(predset)
predset[0:10]

[(1, 318, 3.579442714350294),
 (1, 1704, 3.579442714350294),
 (1, 6874, 3.579442714350294),
 (1, 8798, 3.579442714350294),
 (1, 46970, 3.579442714350294),
 (1, 48516, 3.579442714350294),
 (1, 58559, 3.579442714350294),
 (1, 60756, 3.579442714350294),
 (1, 68157, 3.579442714350294),
 (1, 71535, 3.579442714350294)]

In [ ]:
predictions = gs_model.test(predset)

In [ ]:
predictions[0:10]

[Prediction(uid=1, iid=318, r_ui=3.579442714350294, est=4.953888892997183, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=1704, r_ui=3.579442714350294, est=4.811858760280403, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=6874, r_ui=3.579442714350294, est=4.745184479485489, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=8798, r_ui=3.579442714350294, est=4.6102450867763745, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=46970, r_ui=3.579442714350294, est=4.139481333486526, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=48516, r_ui=3.579442714350294, est=4.9620836959599846, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=58559, r_ui=3.579442714350294, est=5.009553009135706, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=60756, r_ui=3.579442714350294, est=4.232274503384751, details={'actual_k': 40

In [ ]:
predictions_df = pd.DataFrame(predictions) ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique() ### promedio de ratings
predictions_df.sort_values(by='est',ascending=False)

,uid,iid,r_ui,est,details
103970,53,318,3.579443,5.745741,"{'actual_k': 20, 'was_impossible': False}"
104487,53,140110,3.579443,5.668647,"{'actual_k': 1, 'was_impossible': False}"
104310,53,750,3.579443,5.637350,"{'actual_k': 19, 'was_impossible': False}"
103932,53,2959,3.579443,5.630968,"{'actual_k': 20, 'was_impossible': False}"
104046,53,904,3.579443,5.593950,"{'actual_k': 20, 'was_impossible': False}"
...,...,...,...,...,...
5477,3,3451,3.579443,1.000000,"{'actual_k': 15, 'was_impossible': False}"
880430,442,158,3.579443,1.000000,"{'actual_k': 20, 'was_impossible': False}"
4326,3,2770,3.579443,1.000000,"{'actual_k': 24, 'was_impossible': False}"
4114,3,2054,3.579443,1.000000,"{'actual_k': 24, 'was_impossible': False}"


In [ ]:
def recomendaciones(user_id,n_recomend=10):

    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendados = predictions_userID[['iid','est']]
    recomendados.to_sql('reco',conn,if_exists="replace")

    recomendados=pd.read_sql('''select a.*, b.title
                             from reco a left join df_final b
                             on a.iid=b.movie_id ''', conn)

    return(recomendados)



recomendaciones(user_id=500,n_recomend=600)

,index,iid,est,title
0,995313,1221,4.191539,"Godfather: Part II, The (1974)"
1,995313,1221,4.191539,"Godfather: Part II, The (1974)"
2,995313,1221,4.191539,"Godfather: Part II, The (1974)"
3,995313,1221,4.191539,"Godfather: Part II, The (1974)"
4,995313,1221,4.191539,"Godfather: Part II, The (1974)"
...,...,...,...,...
30771,995480,116797,3.518385,The Imitation Game (2014)
30772,995480,116797,3.518385,The Imitation Game (2014)
30773,995480,116797,3.518385,The Imitation Game (2014)
30774,995480,116797,3.518385,The Imitation Game (2014)
